In [19]:
import robotic as ry
import numpy as np

from time import sleep

import math
import sys


r1 = "r1"
r2 = "r2"
m1 = "m1"
m2 = "m2"
g1 = "g1"
g2 = "g2"

robots = [r1, r2]
m_objects = [m1, m2]
goals = [g1, g2]

goals_reverse = [g2, g1]

def komo_solver(robots, m_objects, goals):
    
    if not isinstance(robots, list):
        robots = [robots]
    if not isinstance(m_objects, list):
        m_objects = [m_objects]
    if not isinstance(goals, list):
        goals = [goals]
    
    komo = ry.KOMO(C, phases=2, slicesPerPhase=20, kOrder=2, enableCollisions=True)  
    komo.addControlObjective([], 0)
    komo.addControlObjective([], 2)
    
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)     
    
    for i in range(len(robots)):
        komo.addObjective([1, -1], ry.FS.distance, [robots[i], m_objects[i]], ry.OT.eq) 
        komo.addModeSwitch([1,-1], ry.SY.stable, [robots[i], m_objects[i]], True) 
        komo.addObjective([2] , ry.FS.positionDiff, [m_objects[i], goals[i]], ry.OT.eq) 

    ret = ry.NLP_Solver(komo.nlp(), verbose=4).solve() 
    
    return (komo, ret)

def komo_onestate_hold(C, robot, m_object):
    komo = ry.KOMO(C, phases=1, slicesPerPhase=1, kOrder=0, enableCollisions=True)
    komo.addControlObjective([], 0)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq) 
    komo.addObjective([], ry.FS.distance, [robot, m_object], ry.OT.eq)
    
    ret = ry.NLP_Solver(komo.nlp(), verbose= 4).solve()
    return (komo, ret)


def komo_onestate_move(C, robot, m_object, goal):
    komo = ry.KOMO(C, phases=1, slicesPerPhase=1, kOrder= 1, enableCollisions=True)
    komo.addModeSwitch([0,1], ry.SY.stable, [robot, m_object], True) 
    komo.addObjective([], ry.FS.distance, [robot, m_object], ry.OT.eq)
    komo.addObjective([1, 2] , ry.FS.positionDiff, [m_object, goal], ry.OT.eq) 
    
    ret = ry.NLP_Solver(komo.nlp(), verbose= 4).solve()
    return (komo, ret)

    

def rrt_solver(k, start_state, end_state, trial_count):
    rrt = ry.PathFinder()
    rrt.setProblem(k, start_state, end_state)
    
    ret = rrt.solve()
    
        
    for i in range(trial_count):
        if ret.feasible:
            return ret
        ret = rrt.solve() 
        print(ret.feasible) 
    
    
    
    return ret


def rrt_viewer(C, ret, attach_state = None, attach = False):
    
    sleep(1)
    if not ret.feasible:
        return
    
    path = ret.x
    
    initial_state = C.getFrameState()
    
    if attach:
        
        C.setJointState(attach_state)
        C.attach(r1, m1)
        C.attach(r2, m2)
    
    
    for i in path:
        C.setJointState(i)
        C.view()
        sleep(0.3)
    
    C.setFrameState(initial_state)
    
    if attach:
        C.getFrame(m1).unLink()
        C.getFrame(m2).unLink()
        
def path_viewer(C, path, attach_index1, attach_index2):
    
    initial_state = C.getFrameState()
    
    C.view()

    sleep(2)
    
    for i in range(0, len(path)):
        
        
        C.setJointState(path[i])
        C.view()
        sleep(0.05)
        
        if i == attach_index1:
            C.attach(r1, m1)
        if i == attach_index2:
            C.attach(r2, m2)
            
    C.setFrameState(initial_state)
    
    C.getFrame(m1).unLink()
    C.getFrame(m2).unLink()
        
        
        

    
    

C = ry.Config()

maps = ["map1_simple_cross.g", "map2_tunnel.g", "map3_two_cave.g", "map4_one_cave.g", "map5_pioneer.g"]

C.addFile(f"maps/{maps[0]}")
C.view()











def path_mapper(path, final_size):
    original_path, hold_index = path
    original_path = np.array(original_path)
    path_len = len(original_path)

    # Sample evenly spaced indices, reserving one for final state and one for hold
    steps_to_sample = final_size - 2  # 1 for hold, 1 for final
    sample_indices = np.linspace(0, path_len - 2, steps_to_sample).astype(int)
    sampled_path = original_path[sample_indices]

    # Prepare hold and final states
    
    final_state = original_path[-1].reshape(1, -1)
    
    insert_pos = -1
    
    if hold_index > 0:
        hold_state = original_path[hold_index].reshape(1, -1)
        insert_pos = np.searchsorted(sample_indices, hold_index)
        with_hold = np.insert(sampled_path, insert_pos, hold_state, axis=0)
        final_result = np.vstack([with_hold, final_state])

    else:
        final_result = np.vstack([sampled_path, final_state])
        
    print("final result: ", final_result)
            
    return final_result, insert_pos


def initial_path_finding(C, serial_no):
    temp = ry.Config()
    temp.addConfigurationCopy(C)
    
    temp.delFrame(f"r{3 - serial_no}")
    temp.delFrame(f"m{3 - serial_no}")
    temp.delFrame(f"g{3 - serial_no}")
    
    r1 = f"r{serial_no}"
    m1 = f"m{serial_no}" 
    g1 = f"g{serial_no}"
    
    
    initial_state = temp.getJointState()
        
    first_robot_hold, ret1 = komo_onestate_hold(temp, r1, m1)  
    j1_hold = first_robot_hold.getPath()[0]  # birinci tobotun cargonun yanına geldiği state

    first_robot_move, ret1 = komo_onestate_move(temp, r1, m1, g1)
    j1_moved = first_robot_move.getPath()[0] # birinci robotun goal yanına geldiği state
    
    
    r1_hold_path = rrt_solver(temp, initial_state, j1_hold, 3)
    r1_move_path = rrt_solver(temp, j1_hold, j1_moved, 3)
    r1_final = (np.array([i for i in np.concatenate((r1_hold_path.x, r1_move_path.x))]), len(r1_hold_path.x))
        
    return r1_final





    




In [20]:
r1_final = path_mapper(initial_path_finding(C, 1), 60)
r2_final = path_mapper(initial_path_finding(C, 2), 60)

final result:  [[ 2.50000000e+00 -3.50000000e+00  0.00000000e+00]
 [ 2.46326573e+00 -3.40699176e+00 -2.47628928e-04]
 [ 2.38159438e+00 -3.22575232e+00 -1.38262577e-02]
 [ 2.33558799e+00 -3.06431290e+00 -3.43298624e-02]
 [ 2.26159869e+00 -2.87851453e+00 -3.21984709e-02]
 [ 2.28135716e+00 -2.70322699e+00  1.22012375e-02]
 [ 2.28604375e+00 -2.52746042e+00 -2.62659708e-02]
 [ 2.24820276e+00 -2.33925536e+00 -3.25088277e-02]
 [ 2.13679075e+00 -2.17317476e+00 -3.16221787e-02]
 [ 2.09008474e+00 -2.03444291e+00 -3.08614539e-03]
 [ 1.99348838e+00 -1.85969431e+00 -1.10153346e-02]
 [ 1.87830604e+00 -1.82560353e+00  1.41199182e-03]
 [ 1.78385693e+00 -1.65764645e+00 -2.72998441e-02]
 [ 1.74767147e+00 -1.46990433e+00 -5.18996410e-02]
 [ 1.68199203e+00 -1.28105498e+00 -5.58050067e-02]
 [ 1.61299291e+00 -1.09337142e+00 -5.93561292e-02]
 [ 1.50754494e+00 -9.37380128e-01 -2.32169893e-02]
 [ 1.41826226e+00 -7.61126470e-01 -7.29208876e-03]
 [ 1.34479372e+00 -5.75109997e-01 -7.78734661e-03]
 [ 1.30805945e+0

In [21]:
# heuristic one -- checking for any collision


def collision_states(r1_path, r2_path): 
    
    state_indices = []
    
    for i in range(0, len(r1_path)):
        
        r1_loc = r1_path[i]
        r2_loc = r2_path[i]
    
        distance = math.sqrt(pow(abs(r1_loc[0] - r2_loc[0]), 2) + pow(abs(r1_loc[1] - r2_loc[1]), 2))
                    
        if distance < 1.1:
            state_indices.append(i)
            
    return state_indices



collision_indices = collision_states(r1_final[0], r2_final[0])

print(collision_indices)

if len(collision_indices) == 0:
    print("problem solved .... path viewer ")
    final_path = np.hstack((r1_final[0], r2_final[0]))    
    path_viewer(C, final_path, r1_final[1], r2_final[1])
    
    sys.exit()
    
def find_another_path_in_collision(collision_states, r1_final, r2_final):
    
    r1_path = r1_final[0]
    r2_path = r2_final[0]
    
    print("r1path===========", r1_path[0])
    
    temp = ry.Config()
    temp.addConfigurationCopy(C)
    
    sstt = np.concatenate((r1_final[0][r1_final[1]], r2_final[0][r2_final[1]]))
    temp.setJointState(sstt)
    
    temp.attach(r1, m1)
    temp.attach(r2, m2)
        
    rrt_again_initial = np.concatenate((r1_path[collision_states[0] - 3], r2_path[collision_states[0] - 3]))
    rrt_again_last = np.concatenate((r1_path[collision_states[-1] + 3], r2_path[collision_states[-1] + 3]))
    
    
    
    rrt_again = rrt_solver(temp, rrt_again_initial, rrt_again_last, 5)
    
    current_path = path_mapper((rrt_again.x, -1), len(collision_indices) + 7)
    
    del temp
    return current_path


def manipulate_main(r1_final, r2_final, collision_indices, current_path):
    r1_path, r1_hold_idx = r1_final
    r2_path, r2_hold_idx = r2_final

    # Compute range to overwrite
    start = max(collision_indices[0] - 3, 0)
    end = min(collision_indices[-1] + 3, min(len(r1_path), len(r2_path), len(current_path[0]) + start + 1))  # assumes r1, r2 same length
    
    print("start: ", start)
    print("end:", end)
    print("len:", len(current_path[0]))

    # Replace values in r1 and r2 from current_path
    r1_path[start:end] = current_path[0][0:end - start, :3]   # First 3 values
    r2_path[start:end] = current_path[0][0:end - start, 3:]   # Last 3 values

    return (r1_path, r1_hold_idx), (r2_path, r2_hold_idx)


def make_wait(r1_final, r2_final, collision_indices):
    r1_path, r1_hold_idx = r1_final
    r2_path, r2_hold_idx = r2_final

    # How many steps to repeat
    wait_len = int(len(collision_indices) / 2) + 3
    insert_idx = collision_indices[0] - 3

    # Get the state to repeat in r1
    wait_state_r1 = r1_path[insert_idx]
    repeated_r1 = np.repeat(wait_state_r1.reshape(1, -1), wait_len, axis=0)

    # Insert wait_state into r1_path at the collision start index
    r1_path = np.insert(r1_path, insert_idx, repeated_r1, axis=0)

    # Extend r2_path by repeating the last element wait_len times
    last_r2 = r2_path[-1]
    repeated_r2 = np.repeat(last_r2.reshape(1, -1), wait_len, axis=0)
    r2_path = np.concatenate((r2_path, repeated_r2), axis=0)

    return (r1_path, r1_hold_idx), (r2_path, r2_hold_idx)




newPath = find_another_path_in_collision(collision_indices, r1_final, r2_final)

if False:
    
    print("wait wait wait ")
    r1_final, r2_final = make_wait(r1_final, r2_final, collision_indices)
    
    print(r1_final[0])
    
    
    newCollisiion = collision_states(r1_final[0], r2_final[0])
    
    if True:
        print("problem solved via waiting")
        final_path = np.hstack((r1_final[0], r2_final[0]))
        
        path_viewer(C, final_path, r1_final[1], r2_final[1])
        
        sys.exit()
    

else:
    
    print("choosing different path")
    
    r1_final, r2_final = manipulate_main(r1_final, r2_final, collision_indices, newPath)
    final_path = np.hstack((r1_final[0], r2_final[0]))
    path_viewer(C, final_path, r1_final[1], r2_final[1])
    sys.exit("problem solved  by selecting differetn collision path")




    
            
        
        
        
    
    


[28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]
r1path=========== [ 2.5 -3.5  0. ]
final result:  [[ 5.31937395e-01 -7.31005256e-01 -3.34237927e-02 -1.22091155e+00
  -4.98220783e-01  5.41344486e-02]
 [ 4.23493808e-01 -9.06617144e-01 -6.71213879e-02 -1.39217277e+00
  -6.42957417e-01  6.35157106e-02]
 [ 2.13179575e-01 -9.71173467e-01  6.73305295e-03 -1.57839313e+00
  -7.08810418e-01  1.00004227e-01]
 [-1.93501508e-02 -1.00468003e+00 -4.16267044e-04 -1.62582407e+00
  -7.24385103e-01  3.60565591e-02]
 [-2.48572953e-01 -8.97497875e-01 -2.17635149e-02 -1.65334315e+00
  -5.54309917e-01  1.55119796e-02]
 [-5.93612171e-01 -9.69185475e-01  1.76788093e-02 -1.62174928e+00
  -4.66214125e-01  2.10803748e-03]
 [-8.19528511e-01 -9.61958536e-01 -1.14688509e-02 -1.63360253e+00
  -1.78052570e-01 -5.67159757e-02]
 [-1.04131293e+00 -9.74975849e-01  1.34683847e-02 -1.56867652e+00
   5.14849658e-02 -8.20566368e-02]
 [-1.17685600e+00 -7.69589795e-01 -3.42560530e-02 -1.37803119e+00
   

SystemExit: problem solved  by selecting differetn collision path

In [ ]:
#path_viewer(C, final_path, r1_final[1], r2_final[1])